# LangChain Basics


In [5]:
#!pip install -r ./requirements.txt -q

In [6]:
#!pip show langchain

In [7]:
#!pip install langchain --upgrade -q

### Python-dotenv


In [8]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

#os.environ.get('PINECONE_API_KEY')

True

### LLM Models (Wrappers): GPT-3

In [9]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [10]:
output = llm('Explain quantum mechanics in one sentence')
print(output)



Quantum mechanics is a branch of physics that describes the behavior of matter on the atomic and subatomic level.


In [11]:
print(llm.get_num_tokens('Explain quantum mechanics in one sentence'))

7


In [12]:
output = llm.generate(['... is the capital of France', 
                   'What is the formula for the area of the circle'])
print(output.generations)

[[Generation(text='\nParis', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='?\n\nThe formula for the area of a circle is A = πr^2, where r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [13]:
# Answer of the second question
print(output.generations[1][0].text)

?

The formula for the area of a circle is A = πr^2, where r is the radius of the circle.


In [14]:
len(output.generations)

2

In [15]:
output = llm.generate(['Write an original tagline for burger restaurant']*3)

In [16]:
for o in output.generations:
    print(o[0].text)



"Taste the Difference at Burger Heaven!"


"Sink your teeth into a delicious burger experience!"


"Taste the Juiciest Burgers in Town!"


### ChatModels: GPT-3.5-Turbo and GPT-4

In [17]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [19]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in Spanish'),
    HumanMessage(content='Explain quantum mechanics in one sentence')
]
output = chat(messages)

In [20]:
print(output)

content='La mecánica cuántica es una teoría física que describe el comportamiento de las partículas subatómicas y sus interacciones a través de principios como la superposición, el entrelazamiento y la dualidad onda-partícula.'


### Prompt Templates

In [22]:
from langchain import PromptTemplate

In [24]:
template = '''You are an experienced virologist. Write a few sentences about the following {virus} in {language}'''
prompt = PromptTemplate(
    input_variables=['virus','language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] template='You are an experienced virologist. Write a few sentences about the following {virus} in {language}'


In [25]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='ebola', language='spanish'))

In [26]:
print(output)



Como experto en la materia de la virología, sé que el ébola es una enfermedad viral grave que causa fiebre hemorrágica y se transmite a través de contacto con fluidos corporales de personas infectadas con el virus. Se ha descubierto que el ébola se origina en África Occidental, pero también se ha reportado en otros países. Es importante tomar precauciones para prevenir la propagación de este virus, como evitar el contacto con personas enfermas y el uso de medidas de protección adecuadas.


### Simple Chains

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

template = '''You are an experienced virologist. Write a few sentences about the following {virus} in {language}'''

prompt = PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus':'HSV', 'language':'spanish'})

In [29]:
print(output)

El HSV, o virus del herpes simple, es un virus altamente contagioso que afecta principalmente a los humanos. Hay dos tipos de HSV: el HSV-1, que generalmente causa herpes labial y ocasionalmente herpes genital, y el HSV-2, que es la principal causa de herpes genital. Estos virus se transmiten a través del contacto directo con una persona infectada o a través de objetos contaminados. Aunque no hay cura para el HSV, existen tratamientos antivirales que pueden ayudar a reducir la frecuencia y la gravedad de los brotes.


### Sequencial Chains

In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an expereinced scientist and Python programmer. 
        Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1,chain2], verbose=True)
output = overall_chain.run('linear regression')



> Entering new SimpleSequentialChain chain...


def linear_regression(x_values, y_values):
    """
    A function that implements the concept of linear regression. 
    Parameters: 
    x_values (list): A list of x values 
    y_values (list): A list of y values
    
    Returns: 
    (float, float): The slope and intercept of the regression line
    """
    x_mean = sum(x_values) / len(x_values)
    y_mean = sum(y_values) / len(y_values)
    
    numerator = 0
    denominator = 0
    for i in range(0, len(x_values)):
        numerator += (x_values[i] - x_mean) * (y_values[i] - y_mean)
        denominator += (x_values[i] - x_mean) ** 2
    
    slope = numerator / denominator
    intercept = y_mean - slope * x_mean
    
    return (slope, intercept)
The `linear_regression` function is a Python function that implements the concept of linear regression. It takes in two parameters, `x_values` and `y_values`, which are both lists of numerical values. These lists represent the x and y coo